In [1]:
import gymnasium as gym
import random
import numpy as np
from collections import deque

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam

In [3]:
env = gym.make("CartPole-v1", render_mode="rgb_array_list")

In [4]:
env.reset()

(array([0.02390552, 0.03067382, 0.02488858, 0.04166652], dtype=float32), {})

In [16]:
import matplotlib.pyplot as plt

In [57]:
%matplotlib inline

In [26]:
#env1 = gym.make("CartPole-v1", render_mode="rgb_array")
env.reset()
for _ in range(1000):
    x = random.randint(0,1)
    observation, reward, terminated, truncated, info = env.step(x)
    print(terminated, truncated, terminated | truncated)
    # print(env.observation_space.sample())
    env.render()
    
env.close()

False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
False False False
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False True
True False Tru

KeyboardInterrupt: 

In [49]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
for e in range(50): #epsiodes
    observation = env.reset()
    
    for t in range(100):
        action = env.action_space.sample() # random value in action spcae of the env
        observation, reward, done, info, _ = env.step(action)
        env.render()
        print(reward)
        if done:
            print("Episode #{} of {}: score {}".format(e, 50, t))
            break
        
print("all episode done")
env.close()

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Episode #0 of 50: score 24
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Episode #1 of 50: score 20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Episode #2 of 50: score 20
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Episode #3 of 50: score 14
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Episode #4 of 50: score 13
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Episode #5 of 50: score 13
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Episode #6 of 50: score 33
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
Episode #7 of 50: score 24
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [141]:
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset(seed=42)
for _ in range(1000):
    action = env.action_space.sample()  # this is where you would insert your policy
    observation, reward, terminated, truncated, info = env.step(action)
    
    if terminated or truncated:
        observation, info = env.reset()
env.close()

`Exploration`: try out different/random things -> good in beginning.


`Exploitation`: sample/collect good experience from the past (exploration) -> good when you have explored a bit

In [52]:
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size # state/observation size is the dimension in which the state of the agent can be represented
        self.action_size = action_size # different type of action/step we can take in the environment
        self.memory = deque(maxlen=2000) # this deque (doubly ended queue) will store the random + best actions agent performs
        self.gamma = 0.95 # discounted rate at which we penalize the later state rewards
        
        self.learning_rate = 1e-4
        self.epsilon = 1.0 # 100% random exploration
        self.epsilon_decay = 0.995 # rate at which random exploration should decay/reduce
                                ## inversaly, rate at which we should start collec
        self.epsilon_min = 0.01
        self.model = self._create_model()
    
    def _create_model(self):
        model = Sequential()
        model.add(Input(4))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(2, activation='linear')) ## 2 is no of possible action
        model.compile(optimizer=Adam(learning_rate=0.001), loss="mse")
        return model
        #model.summary()
        
    def store(self, state, action, reward, gameover, next_state):
        self.memory.append((state, action, reward, gameover, next_state))
    
    def action(self, state):
        
        # for epsilon % of time agent take random action 
        if np.random.rand() <= self.epsilon:
            return env.action_space.sample()
        
        # agent take action using neural netwok brain
        else:
            return np.argmax(self.model.predict(state, verbose=0, use_multiprocessing=True)[0]) # argmax returns the action (index) of max target
    
    def train(self, buffer_size=32):
        # training using "replay buffer" i.e., take a batch of data/experience from memory
        buffer = random.sample(self.memory, buffer_size)
        
        for experience in buffer:
            state, action, reward, gameover, next_state = experience
            # X, Y: state, expected/predicted reward
            
            if not gameover:
                # game is not over - use bellman eqn to find approx target value of reward
                # print(self.model.predict(next_state)[0])
                target = reward + self.gamma * np.argmax(self.model.predict(next_state, verbose=0, use_multiprocessing=True)[0])
            else:
                # when game is over
                target = reward
                
            #print(state)
            target_f = self.model.predict(state, verbose=0)
            # print(type(target_f), action, target, target_f)
            target_f[0, action] = target
            
            self.model.fit(state, target_f, epochs=1, verbose=0, use_multiprocessing=True)
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def laod(self, name):
        self.model.load_weights(name)
        
    def save(self, name):
        self.model.save_weights(name)

`Play with environment and training our model`

In [40]:
n_episodes = 1_000
output_dir = "carpole_model/"

In [41]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
x = env.reset()

In [42]:
# create a instance of agent 
state_size=4
action_size=2
agent = Agent(state_size, action_size)
done = False

In [43]:
# let the agent play the game
for e in range(n_episodes):
    state = env.reset()[0]
    for time in range(500):
        action = agent.action(state) # agent deciding what button to press
        next_state, reward, terminated, truncated, info = env.step(action)
        gameover = terminated | truncated
        # penalize reward when gameover
        if gameover: reward = -10
        # formatting next_state to store in agent memory as the NN (brain) can process np array
        state = state.reshape((1, state_size))
        next_state = next_state.reshape((1, state_size))
        # fed the agent brain with the data
        agent.store(state, action, reward, gameover, next_state)
        
        if gameover:
            print("Episode #{} of {} score: {} exploration rate: {:.2}".format(e, n_episodes, time, agent.epsilon))
            break
        
        if len(agent.memory) > 32:
            agent.train()
            
        if e%50 == 0:
            agent.save(output_dir+"weights_"+'%.4f' %e + ".hdf5")
        
print("model trained")

Episode #0 of 1000 score: 8 exploration rate: 1.0
Episode #1 of 1000 score: 17 exploration rate: 1.0
Episode #2 of 1000 score: 27 exploration rate: 0.78


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_7/dense_21/MatMul' defined at (most recent call last):
    File "Z:\Google\Anaconda\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "Z:\Google\Anaconda\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "Z:\Google\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "Z:\Google\Anaconda\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
      app.start()
    File "Z:\Google\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
      self.io_loop.start()
    File "Z:\Google\Anaconda\lib\site-packages\tornado\platform\asyncio.py", line 149, in start
      self.asyncio_loop.run_forever()
    File "Z:\Google\Anaconda\lib\asyncio\base_events.py", line 570, in run_forever
      self._run_once()
    File "Z:\Google\Anaconda\lib\async

In [281]:
agent.memory[0] # state, action, reward, gameover, next_state

(array([[0.02175644, 0.04204665, 0.02615108, 0.0371155 ]], dtype=float32),
 0,
 array([[ 0.02259737, -0.15344036,  0.02689339,  0.3379333 ]],
       dtype=float32),
 1.0,
 False)

In [259]:
x = np.array([[-0.00822658, -0.01690743]])


In [286]:
print("<class 'numpy.ndarray'> 0 [[0.01952857 0.04128878 0.03365205 0.05385091]] [[-0.0052758   0.00259107]]")

<class 'numpy.ndarray'> 0 [[0.01952857 0.04128878 0.03365205 0.05385091]] [[-0.0052758   0.00259107]]
